In [1]:
#aiko package
#Denis Titov
#16.01.2019

"""
aiko gives read/write functionality for a .hdf5 database.
so far it only works on that specific database :/
*path*/snapdir_000/snapshot_000.0.hdf5
"""

import h5py
import numpy as np
import sys
import os
import pandas as pd
import re

In [29]:
#class for reading snapshot directories, managed by aiko
class snapshot_read_raw():
    #sets path of snapshot dir (corrects for missing slash at end of path) and other vars
    def __init__(self, snapPath, blockNames, particleTypes):
        if snapPath[-1] is not "/":
            snapPath = snapPath + "/"
        self.snapPath = snapPath
        self.blockNames = blockNames
        self.particleTypes = particleTypes
        self.particleTypeIndexes = self.get_particleType_index()
        self.numSnapdirs = self.get_numSnapdirs()
        self.currentSnapshot = self.get_snapshot()

    #searches for a file with correct name, opens and gets number of snaps from header
    def get_numSnapdirs(self):
        for i in os.listdir(self.snapPath):
            if re.search("\.\d\.", i):
                fileName = i
                break
        filePath = self.snapPath + fileName
        file = h5py.File(filePath, 'r')
        attrs = file['/Header'].attrs
        numFilesPerSnapshot = attrs["NumFilesPerSnapshot"]
        return numFilesPerSnapshot

    #converts words of particleTypes to particleTypeIndexes
    def get_particleType_index(self):
        indexList = []
        index = {"Gas":0, "DM":1, "Darkmatter":1, "Stars":4}
        for particleType in self.particleTypes:
            converted = index[particleType]
            indexList.append(converted)
        return indexList

    #returns current snapshot
    def get_snapshot(self):
        for i in os.listdir(self.snapPath):
            if re.search("\_\d{3}\.", i):
                snapshot = re.findall("\_\d{3}\.", i)[0][1:4]
                return int(snapshot)

    #returns list of all filenames in dir
    def get_snapdirList(self):
        return os.listdir(self.snapPath)

    #returns dataframe of snap-particledata
    def read_arepo_snap(self):
        dfSnapList = []
        for filename in self.get_snapdirList():
            file = h5py.File(self.snapPath +  filename, "r")
            dfFileList = []
            typeList = list(file)
            print("reading " + filename)
            for particleTypeIndex in self.particleTypeIndexes:
                typeStructure = "PartType{}".format(particleTypeIndex)
                if typeStructure not in typeList:
                    break
                dfStructure = {}
                for blockName in self.blockNames:
                    h5Path = "/PartType{}/{}".format(particleTypeIndex, blockName)
                    if blockName == "Coordinates":
                        coords = np.float32(file[h5Path].value)
                        dfStructure["posX"] = coords[:,0]
                        dfStructure["posY"] = coords[:,1]
                        dfStructure["posZ"] = coords[:,2]
                    if blockName == "Velocities":
                        vels = np.float32(file[h5Path].value)
                        dfStructure["velX"] = vels[:,0]
                        dfStructure["velY"] = vels[:,1]
                        dfStructure["velZ"] = vels[:,2]
                    if blockName == "Density" and particleTypeIndex == 0:
                        dens = np.float32(file[h5Path].value)
                        dfStructure["Dens"] = dens
                ID = np.uint32(file["/PartType{}/{}".format(particleTypeIndex, "ParticleIDs")].value)
                dfStructure["ID"] = ID
                dfStructure["Snap"] = ID*0+self.currentSnapshot
                dfStructure["Type"] = ID*0+particleTypeIndex
                dfCurrent = pd.DataFrame(dfStructure)
                dfFileList.append(dfCurrent)
            dfFile = pd.concat(dfFileList, sort=True)
            dfSnapList.append(dfFile)
        dfSnap = pd.concat(dfSnapList, ignore_index=True)
        return dfSnap

#{"x":pos[:,0],"y":pos[:,1],"z":pos[:,2],"dens":dens , "ID":ID, "snap":ID*0+s}
#pos = np.float32(f['/PartType0/Coordinates'].value)
#a = snapshot_read(snapPath="/store/clues/HESTIA/RE_SIMS/2048/GAL_FOR/37_11/output/snapdir_122", particleTypes=["Gas", "Stars"], blockNames=["Coordinates", "Velocities", "Density"])

#dfa = a.read_arepo_snap()

#dfa

In [42]:
#class for reading snapshot galaxy data, managed by aiko
class snapshot_read_gal():
    #sets path of snapshot dir (corrects for missing slash at end of path) and other vars
    def __init__(self, groupPath, blockNames):
        if groupPath[-1] is not "/":
            groupPath = groupPath + "/"
        self.groupPath = groupPath
        self.blockNames = self.convert_blockNames(blockNames)
        self.numGroupdirs = self.get_numGroupdirs()
        self.currentSnapshot = self.get_snapshot()

    #searches for a file with correct name, opens and gets number of snaps from header
    def get_numGroupdirs(self):
        return len(os.listdir(self.groupPath))

    #returns current snapshot
    def get_snapshot(self):
        for i in os.listdir(self.groupPath):
            if re.search("\_\d{3}\.", i):
                snapshot = re.findall("\_\d{3}\.", i)[0][1:4]
                return int(snapshot)

    #returns list of all filenames in dir
    def get_groupdirList(self):
        return os.listdir(self.groupPath)

    #returns converted list of grouppath
    def convert_blockNames(self, blockNames):
        newBlock = []
        if "Velocities" in blockNames:
            newBlock.append("GroupVel")
        if "CenterOfMass" in blockNames:
            newBlock.append("GroupCM")
        if "Radius" in blockNames:
            newBlock.append("Group_R_Crit200")
        if "Coordinates" in blockNames:
            newBlock.append("GroupPos")
        return newBlock

    #returns dataframe of snap-particledata
    def read_arepo_gal(self):
        dfFileList = []
        for filename in self.get_groupdirList():
            file = h5py.File(self.groupPath +  filename, "r")
            print("reading " + filename)
            dfStructure = {}
            for blockName in self.blockNames:
                h5Path = "/Group/{}".format(blockName)
                if blockName == "GroupPos":
                    coords = np.float32(file[h5Path].value)
                    dfStructure["posX"] = coords[:,0]
                    dfStructure["posY"] = coords[:,1]
                    dfStructure["posZ"] = coords[:,2]
                if blockName == "GroupVel":
                    vels = np.float32(file[h5Path].value)
                    dfStructure["velX"] = vels[:,0]
                    dfStructure["velY"] = vels[:,1]
                    dfStructure["velZ"] = vels[:,2]
                if blockName == "GroupCM":
                    CM = np.float32(file[h5Path].value)
                    dfStructure["CMX"] = CM[:,0]
                    dfStructure["CMY"] = CM[:,1]
                    dfStructure["CMZ"] = CM[:,2]
                if blockName == "Group_R_Crit200":
                    radius = np.float32(file[h5Path].value)
                    dfStructure["Radius"] = radius
            #ID = np.uint32(file["/Group/{}".format(particleTypeIndex, "ParticleIDs")].value)
            dfStructure["Snap"] = radius*0+self.currentSnapshot
            dfCurrent = pd.DataFrame(dfStructure)
            dfFileList.append(dfCurrent)
        dfSnap = pd.concat(dfFileList, ignore_index=True)
        return dfSnap



a = snapshot_read_gal(groupPath="/store/clues/HESTIA/RE_SIMS/2048/GAL_FOR/37_11/output/groups_124", blockNames=["Velocities", "CenterOfMass", "Radius", "Coordinates"])

a.read_arepo_gal()

reading fof_subhalo_tab_124.0.hdf5
reading fof_subhalo_tab_124.7.hdf5
reading fof_subhalo_tab_124.5.hdf5
reading fof_subhalo_tab_124.4.hdf5
reading fof_subhalo_tab_124.2.hdf5
reading fof_subhalo_tab_124.1.hdf5
reading fof_subhalo_tab_124.6.hdf5
reading fof_subhalo_tab_124.3.hdf5


,velX,velY,velZ,CMX,CMY,CMZ,Radius,posX,posY,posZ,Snap
0,-314.279877,417.511688,-122.820061,46.569164,50.579906,48.010319,0.148954,46.576164,50.574932,48.005283,124.0
1,-321.829865,453.721497,-35.380066,46.928230,50.150578,47.816654,0.150754,46.931679,50.146667,47.825478,124.0
2,-281.751434,526.503967,82.369354,43.688469,53.806282,47.873936,0.154345,43.686790,53.807232,47.866470,124.0
3,-501.527771,515.304565,59.869240,43.703861,53.838615,50.981098,0.143780,43.708492,53.833836,50.978672,124.0
4,-249.509827,330.156189,-134.082092,47.392151,48.301945,45.767590,0.122178,47.393108,48.304718,45.767563,124.0
5,-534.775452,571.051392,-90.044823,43.680676,52.461781,52.041904,0.204638,43.661530,52.519695,52.065269,124.0
6,-335.455414,362.333038,-96.452080,46.270264,50.978992,48.612396,0.095068,46.307446,51.012177,48.604263,124.0
7,-405.706726,316.235565,-30.357065,46.780922,50.716331,47.665028,0.097903,46.781902,50.718552,47.663269,124.0
8,-298.233704,532.141357,-35.574100,44.904236,50.504673,50.309853,0.073708,44.946545,50.519554,50.316982,124.0
9,-403.860931,510.570526,-65.540154,45.871178,50.887501,50.502575,0.092463,45.873394,50.882614,50.506058,124.0


In [18]:
#dfa.set_index("ID", inplace=True)
#dfa[40474477]

In [19]:
#dfa.index.values
#dfa.info()

In [50]:
class arepo_reader():
    #defines vars
    def __init__(self, path, snapshotRange, particleTypes=["Gas", "DM", "Darkmatter", "Stars"], blockNames=["Coordinates", "Velocities", "Density"], groupBlockNames=["Velocities", "CenterOfMass", "Radius", "Coordinates"]):
        if path[-1] is not "/":
            path = path + "/"
        self.path = path
        self.snapshots = np.arange(snapshotRange[0], snapshotRange[1]+1, 1)
        self.particleTypes = particleTypes
        self.blockNames = blockNames
        self.groupBlockNames = groupBlockNames

    #gets list of dirnames for each selected snapshot
    def get_path_raw(self, snapshot):
        Snapdir = "snapdir_" + "%03d" % snapshot + "/"
        path = self.path + Snapdir
        return path

    #
    def get_path_gal(self, snapshot):
        groupdir = "groups_" + "%03d" % snapshot + "/"
        path = self.path + groupdir
        return path

    # returns galaxy data in df
    def read_arepo_gal(self):
        dataList = []
        for snapshot in self.snapshots:
            currentPath = self.get_path_gal(snapshot)
            currentInstance = snapshot_read_gal(currentPath, blockNames=self.groupBlockNames)
            currentData = currentInstance.read_arepo_gal()
            dataList.append(currentData)
        print("processing...")
        dfAll = pd.concat(dataList, ignore_index=True)
        #dfAll.set_index("ID", inplace=True)
        print("done!")
        return dfAll

    # returns full list in df
    def read_arepo_raw(self):
        dataList = []
        for snapshot in self.snapshots:
            currentPath = self.get_path_raw(snapshot)
            currentInstance = snapshot_read_raw(currentPath, particleTypes=self.particleTypes, blockNames=self.blockNames)
            currentData = currentInstance.read_arepo_snap()
            dataList.append(currentData)
        print("processing...")
        dfAll = pd.concat(dataList, ignore_index=True)
        dfAll.set_index("ID", inplace=True)
        print("done!")
        return dfAll






b = arepo_reader(path="/store/clues/HESTIA/RE_SIMS/2048/GAL_FOR/37_11/output", snapshotRange=[126, 127], particleTypes=["Stars"], blockNames=["Coordinates"])

df = b.read_arepo_gal()
df

reading fof_subhalo_tab_126.7.hdf5
reading fof_subhalo_tab_126.0.hdf5
reading fof_subhalo_tab_126.2.hdf5
reading fof_subhalo_tab_126.1.hdf5
reading fof_subhalo_tab_126.6.hdf5
reading fof_subhalo_tab_126.4.hdf5
reading fof_subhalo_tab_126.5.hdf5
reading fof_subhalo_tab_126.3.hdf5
reading fof_subhalo_tab_127.1.hdf5
reading fof_subhalo_tab_127.3.hdf5
reading fof_subhalo_tab_127.4.hdf5
reading fof_subhalo_tab_127.2.hdf5
reading fof_subhalo_tab_127.7.hdf5
reading fof_subhalo_tab_127.0.hdf5
reading fof_subhalo_tab_127.5.hdf5
reading fof_subhalo_tab_127.6.hdf5
processing...
done!


,velX,velY,velZ,CMX,CMY,CMZ,Radius,posX,posY,posZ,Snap
0,-310.114014,417.585754,-206.849213,48.004082,47.647289,47.771305,0.008688,48.004456,47.645454,47.775940,126.0
1,-269.071564,404.763733,-234.121323,48.305088,46.273178,47.366692,0.007028,48.299042,46.268856,47.356659,126.0
2,-419.425751,510.827515,-39.757893,45.350372,51.201374,51.121502,0.000000,45.350372,51.201374,51.121502,126.0
3,-298.153503,417.381683,-179.982147,48.888996,46.991219,48.561455,0.000000,48.888996,46.991219,48.561455,126.0
4,-338.573853,441.267456,-123.479095,48.456394,47.627613,49.440475,0.000000,48.456394,47.627613,49.440475,126.0
5,-555.511414,638.970093,-51.824387,43.655529,52.393047,51.938679,0.000000,43.655529,52.393047,51.938679,126.0
6,-451.788635,466.453278,-24.747408,45.119553,51.165253,50.850304,0.010498,45.119003,51.166531,50.852131,126.0
7,-578.888672,674.748779,-61.026482,43.708168,52.571468,51.855221,0.000000,43.708168,52.571468,51.855221,126.0
8,-476.528503,691.626770,97.104561,43.949154,52.545059,51.797127,0.009770,43.950119,52.543453,51.801250,126.0
9,-488.700165,482.809235,-109.798912,43.399529,52.504665,51.808441,0.010136,43.401512,52.503239,51.809540,126.0


In [33]:
df.reset_index(drop=True)
df

Dens 
 ID 
 Snap 
 Type 
 posX 
 posY 
 posZ 
 velX 
 velY 
 velZ 
 
 
 
 
 1 
 0 
 1.326839 
 48946118 
 1 
 0 
 86.061394 
 36.254452 
 89.566231 
 241.205338 
 721.257507 
 228.771378 
 
 
 1 
 1.420396 
 48946119 
 1 
 0 
 86.444191 
 36.240009 
 89.569473 
 239.242325 
 699.268127 
 224.528107 
 
 
 2 
 1.340642 
 48946120 
 1 
 0 
 86.821083 
 36.234314 
 89.570061 
 203.201126 
 676.695251 
 224.037567 
 
 
 3 
 1.300957 
 48946121 
 1 
 0 
 87.218491 
 36.221497 
 89.566406 
 201.726715 
 645.879150 
 215.498917 
 
 
 4 
 1.269144 
 48946122 
 1 
 0 
 75.137260 
 36.548218 
 89.682945 
 257.186432 
 483.666077 
 527.786499 
 
 
 5 
 1.332111 
 48946123 
 1 
 0 
 75.514023 
 36.554264 
 89.670517 
 250.424316 
 511.801636 
 505.088074 
 
 
 6 
 1.360232 
 48946124 
 1 
 0 
 75.915489 
 36.585464 
 89.668335 
 250.964539 
 565.485413 
 490.883087 
 
 
 7 
 1.412399 
 48946125 
 1 
 0 
 76.300995 
 36.597042 
 89.664520 
 250.244034 
 601.172485 
 479.977997 
 
 
 8 
 1.458149 
 48946126 
 1 
 0 
 76.686363 
 36.598534 
 89.654640 
 243.976013 
 617.522766 
 457.125275 
 
 
 9 
 1.485155 
 48946127 
 1 
 0 
 77.057281 
 36.611645 
 89.644012 
 214.129242 
 637.257019 
 429.538727 
 
 
 10 
 1.460380 
 48946128 
 1 
 0 
 77.441292 
 36.606544 
 89.626411 
 188.258362 
 632.444580 
 385.714630 
 
 
 11 
 1.430669 
 48946129 
 1 
 0 
 77.823677 
 36.614651 
 89.612541 
 162.453781 
 633.960754 
 345.996521 
 
 
 12 
 1.474944 
 48946130 
 1 
 0 
 78.203430 
 36.610424 
 89.602150 
 138.727005 
 626.450317 
 318.727509 
 
 
 13 
 1.395441 
 48946131 
 1 
 0 
 78.567230 
 36.603294 
 89.590240 
 91.045586 
 617.652100 
 287.441711 
 
 
 14 
 1.261046 
 48946132 
 1 
 0 
 78.963707 
 36.599617 
 89.568047 
 87.668488 
 605.251892 
 241.961548 
 
 
 15 
 1.236070 
 48946133 
 1 
 0 
 79.371284 
 36.580452 
 89.558342 
 119.491257 
 574.138000 
 213.108429 
 
 
 16 
 1.231733 
 48946134 
 1 
 0 
 79.766243 
 36.572186 
 89.558113 
 134.180206 
 548.496277 
 200.685043 
 
 
 17 
 1.259353 
 48946135 
 1 
 0 
 80.156715 
 36.570972 
 89.552116 
 131.294067 
 534.121948 
 181.770264 
 
 
 18 
 1.244269 
 48946136 
 1 
 0 
 80.539345 
 36.565254 
 89.555939 
 112.639198 
 524.773682 
 179.063232 
 
 
 19 
 1.168302 
 48946137 
 1 
 0 
 80.930916 
 36.564262 
 89.556290 
 114.613846 
 522.494690 
 175.625961 
 
 
 20 
 1.164823 
 48946138 
 1 
 0 
 81.345428 
 36.565769 
 89.549339 
 153.240387 
 526.850281 
 160.641998 
 
 
 21 
 1.262688 
 48946139 
 1 
 0 
 81.749603 
 36.565331 
 89.539314 
 191.702011 
 539.188354 
 148.186356 
 
 
 22 
 1.329544 
 48946140 
 1 
 0 
 82.152565 
 36.584579 
 89.538330 
 213.598221 
 574.932251 
 140.176849 
 
 
 23 
 1.391783 
 48946141 
 1 
 0 
 82.533623 
 36.607246 
 89.528595 
 200.589630 
 632.415100 
 132.461655 
 
 
 24 
 1.471375 
 48946142 
 1 
 0 
 82.919243 
 36.631931 
 89.539993 
 186.058441 
 688.082581 
 151.980148 
 
 
 25 
 1.553750 
 48946143 
 1 
 0 
 83.296478 
 36.640842 
 89.551796 
 164.894318 
 718.931885 
 176.749329 
 
 
 26 
 1.516938 
 48946144 
 1 
 0 
 83.668602 
 36.642464 
 89.559044 
 134.045715 
 724.568848 
 196.355194 
 
 
 27 
 1.497412 
 48946145 
 1 
 0 
 84.078270 
 36.641281 
 89.564903 
 144.338760 
 723.066467 
 217.955246 
 
 
 28 
 1.607197 
 48946146 
 1 
 0 
 84.454330 
 36.637993 
 89.577644 
 118.833305 
 723.984802 
 248.420059 
 
 
 29 
 1.474549 
 48946147 
 1 
 0 
 84.818153 
 36.628677 
 89.589470 
 78.904671 
 711.630676 
 276.247620 
 
 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 
 
 3 
 5143860 
 1.207486 
 41270371 
 3 
 0 
 40.901585 
 74.128105 
 45.649971 
 -347.715515 
 -281.244202 
 -199.830292 
 
 
 5143861 
 1.242470 
 41270372 
 3 
 0 
 41.306934 
 74.123169 
 45.634644 
 -299.611572 
 -303.419647 
 -228.530487 
 
 
 5143862 
 1.241138 
 41270373 
 3 
 0 
 41.700016 
 74.109833 
 45.634048 
 -289.899109 
 -329.535797 
 -235.387939 
 
 
 5143863 
 1.221407 
 41270374 
 3 
 0 
 42.103241 
 74.098267 
 45.

In [35]:
del df["ID"]
df

Dens 
 Snap 
 Type 
 posX 
 posY 
 posZ 
 velX 
 velY 
 velZ 
 
 
 
 
 1 
 0 
 1.326839 
 1 
 0 
 86.061394 
 36.254452 
 89.566231 
 241.205338 
 721.257507 
 228.771378 
 
 
 1 
 1.420396 
 1 
 0 
 86.444191 
 36.240009 
 89.569473 
 239.242325 
 699.268127 
 224.528107 
 
 
 2 
 1.340642 
 1 
 0 
 86.821083 
 36.234314 
 89.570061 
 203.201126 
 676.695251 
 224.037567 
 
 
 3 
 1.300957 
 1 
 0 
 87.218491 
 36.221497 
 89.566406 
 201.726715 
 645.879150 
 215.498917 
 
 
 4 
 1.269144 
 1 
 0 
 75.137260 
 36.548218 
 89.682945 
 257.186432 
 483.666077 
 527.786499 
 
 
 5 
 1.332111 
 1 
 0 
 75.514023 
 36.554264 
 89.670517 
 250.424316 
 511.801636 
 505.088074 
 
 
 6 
 1.360232 
 1 
 0 
 75.915489 
 36.585464 
 89.668335 
 250.964539 
 565.485413 
 490.883087 
 
 
 7 
 1.412399 
 1 
 0 
 76.300995 
 36.597042 
 89.664520 
 250.244034 
 601.172485 
 479.977997 
 
 
 8 
 1.458149 
 1 
 0 
 76.686363 
 36.598534 
 89.654640 
 243.976013 
 617.522766 
 457.125275 
 
 
 9 
 1.485155 
 1 
 0 
 77.057281 
 36.611645 
 89.644012 
 214.129242 
 637.257019 
 429.538727 
 
 
 10 
 1.460380 
 1 
 0 
 77.441292 
 36.606544 
 89.626411 
 188.258362 
 632.444580 
 385.714630 
 
 
 11 
 1.430669 
 1 
 0 
 77.823677 
 36.614651 
 89.612541 
 162.453781 
 633.960754 
 345.996521 
 
 
 12 
 1.474944 
 1 
 0 
 78.203430 
 36.610424 
 89.602150 
 138.727005 
 626.450317 
 318.727509 
 
 
 13 
 1.395441 
 1 
 0 
 78.567230 
 36.603294 
 89.590240 
 91.045586 
 617.652100 
 287.441711 
 
 
 14 
 1.261046 
 1 
 0 
 78.963707 
 36.599617 
 89.568047 
 87.668488 
 605.251892 
 241.961548 
 
 
 15 
 1.236070 
 1 
 0 
 79.371284 
 36.580452 
 89.558342 
 119.491257 
 574.138000 
 213.108429 
 
 
 16 
 1.231733 
 1 
 0 
 79.766243 
 36.572186 
 89.558113 
 134.180206 
 548.496277 
 200.685043 
 
 
 17 
 1.259353 
 1 
 0 
 80.156715 
 36.570972 
 89.552116 
 131.294067 
 534.121948 
 181.770264 
 
 
 18 
 1.244269 
 1 
 0 
 80.539345 
 36.565254 
 89.555939 
 112.639198 
 524.773682 
 179.063232 
 
 
 19 
 1.168302 
 1 
 0 
 80.930916 
 36.564262 
 89.556290 
 114.613846 
 522.494690 
 175.625961 
 
 
 20 
 1.164823 
 1 
 0 
 81.345428 
 36.565769 
 89.549339 
 153.240387 
 526.850281 
 160.641998 
 
 
 21 
 1.262688 
 1 
 0 
 81.749603 
 36.565331 
 89.539314 
 191.702011 
 539.188354 
 148.186356 
 
 
 22 
 1.329544 
 1 
 0 
 82.152565 
 36.584579 
 89.538330 
 213.598221 
 574.932251 
 140.176849 
 
 
 23 
 1.391783 
 1 
 0 
 82.533623 
 36.607246 
 89.528595 
 200.589630 
 632.415100 
 132.461655 
 
 
 24 
 1.471375 
 1 
 0 
 82.919243 
 36.631931 
 89.539993 
 186.058441 
 688.082581 
 151.980148 
 
 
 25 
 1.553750 
 1 
 0 
 83.296478 
 36.640842 
 89.551796 
 164.894318 
 718.931885 
 176.749329 
 
 
 26 
 1.516938 
 1 
 0 
 83.668602 
 36.642464 
 89.559044 
 134.045715 
 724.568848 
 196.355194 
 
 
 27 
 1.497412 
 1 
 0 
 84.078270 
 36.641281 
 89.564903 
 144.338760 
 723.066467 
 217.955246 
 
 
 28 
 1.607197 
 1 
 0 
 84.454330 
 36.637993 
 89.577644 
 118.833305 
 723.984802 
 248.420059 
 
 
 29 
 1.474549 
 1 
 0 
 84.818153 
 36.628677 
 89.589470 
 78.904671 
 711.630676 
 276.247620 
 
 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 
 
 3 
 5143860 
 1.207486 
 3 
 0 
 40.901585 
 74.128105 
 45.649971 
 -347.715515 
 -281.244202 
 -199.830292 
 
 
 5143861 
 1.242470 
 3 
 0 
 41.306934 
 74.123169 
 45.634644 
 -299.611572 
 -303.419647 
 -228.530487 
 
 
 5143862 
 1.241138 
 3 
 0 
 41.700016 
 74.109833 
 45.634048 
 -289.899109 
 -329.535797 
 -235.387939 
 
 
 5143863 
 1.221407 
 3 
 0 
 42.103241 
 74.098267 
 45.635258 
 -273.819794 
 -356.616150 
 -224.795151 
 
 
 5143864 
 1.211174 
 3 
 0 
 42.501141 
 74.089485 
 45.635864 
 -247.599335 
 -380.168884 
 -221.394073 
 
 
 5143865 
 1.188657 
 3 
 0 
 42.911648 
 74.081635 
 45.640717 
 -206.862595 
 -405.869995 
 -221.918503 
 
 
 5143866 
 1.200988 
 3 
 0 
 43.325104 
 74.056290 
 45.633984 
 -157.774719 
 -446.987701 
 -230.246841 
 
